# What is this notebook for?
This notebook is intended only for visualinsing the left and or right ventricle segmentations for a particular echonet video.

If you want to add other interesting things to the visualisation (such as estimated septum width, etc.), you should use the [other notebook](./weak_labels.ipynb) instead.

In [1]:
from pathlib import Path
import sys
import math
from typing import List, Tuple

import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import scipy

import echonet

config = dotenv_values(".env")

# Can assign these colours to numpy arrays so long as the colours are stored in
# the last axis of the target array (e.g. image.shape =(112, 112, 3), but not
# image.shape = (3, 112, 112)). 
# Just do image[y_vals, x_vals] = MAGENTA
# IMPORTANT: if using within an *opencv* function, you'll want to do 
# COLOUR.tolist() to convert these to python primitives, else opencv complains about
# datatypes
# Note also that these are BGR, not RGB, since that's what opencv prefers BGR for historical reasons!
RED = np.array([0, 0, 255])
GREEN = np.array([0, 255, 0])
BLUE = np.array([255, 0, 0])
ORANGE = np.array([0, 165, 255])
LIGHT_GREY = np.array([211, 211, 211])
MAGENTA = np.array([255, 0, 255])
YELLOW = np.array([0, 255, 255])
WHITE = np.array([255, 255, 255])
BLACK = np.array([0, 0, 0])

# Just some types for us to use in type hints to make dev easier
Point = List[np.intp]
Box = Tuple[Point, Point, Point, Point]
Rectangle = Tuple[Point, Tuple[float, float], float] # [centre, (width, height), angle]

In [2]:
WRITE = False

config = dotenv_values(".env")

echonet_video_fp = (Path
(config["ECHONET_VIDEO_DIR"]) / config["VIDEONAME"]).with_suffix(".avi")
echonet_video = echonet.utils.loadvideo(str(echonet_video_fp))
echonet_video = echonet_video.transpose((1, 2, 3, 0)) # Put colour axis at end for easier colouring of pixels

LV_masks = np.load(config["LV_MASKS"])
RV_masks = np.load(config["RV_MASKS"])

num_frames, frame_height, frame_width = LV_masks.shape
out_height = (frame_height + frame_height // 8) * 4
out_width = frame_width * 4
out_size = (out_width, out_height)
print(f"Size: ({frame_width}, {frame_height}) -> ({out_size})")

WINDOW = f"Segmentation: {config['VIDEONAME']}.avi"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)

i = 0
is_playing = True

if WRITE:
    writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'MJPG'), 30, out_size)

try:
    while True:
        if i >= num_frames:
            i = 0
            if WRITE:
                break
        elif i < 0:
            i = num_frames - 1

        # Copy data for this particular frame
        frame = echonet_video[i].copy()
        LV_mask = LV_masks[i].copy()
        RV_mask = RV_masks[i].copy()
        
        ######## SEGMENTATIONS
        frame[LV_mask] = RED
        frame[RV_mask] = BLUE
        
        ####### ADD FRAME COUNTER AT TOP
        top_border = np.zeros((frame_height // 8, frame_width, 3), dtype=frame.dtype)
        top_border[:, :] = np.expand_dims(LIGHT_GREY, (0, 1))
        cv2.putText(top_border, f"Frame {i+1}/{num_frames}", org=(5,10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.25, color=RED.tolist())
        frame = np.concatenate([top_border, frame], axis=0)

        ######## SHOW SEGMENTATION AND HANDLE KEYPRESS
        cv2.imshow(WINDOW, frame)

        if WRITE:
            frame = cv2.resize(frame, out_size)
            writer.write(frame)

        keypress = cv2.waitKey(25) & 0xFF
        if keypress == ord('q'):
            break
        elif keypress == ord(' '):
            is_playing = not is_playing
        elif keypress == ord('a'):
            i -= 1
        elif keypress == ord('d'):
            i += 1
        else:
            if is_playing:
                i += 1
finally:
    cv2.destroyAllWindows() 
    if WRITE:
        writer.release()

Size: (112, 112) -> ((448, 504))


In [6]:
from weak_labels.utils import mask_to_image, get_average_eccentricity
images = mask_to_image(RV_masks)
contours, _ = cv2.findContours(images[0], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
(centre_x, centre_y), (minor_radius, major_radius), angle = cv2.fitEllipse(contours[0])
ell_ecc = np.sqrt(1 - (minor_radius/major_radius)**2)

print("From rectangle", get_average_eccentricity(RV_masks))
print("From ellipse", ell_ecc)

From rectangle 0.8990815108628878
From ellipse 0.9170112007036298
